In [31]:
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import BatchLoader, DisjointLoader
from spektral.datasets import TUDataset
from spektral.layers import GCNConv, GlobalAvgPool, GraphMasking

In [32]:
# physical_devices = tf.config.list_physical_devices("GPU")
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)

################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################
data = TUDataset("PROTEINS")

# Train/test split
np.random.shuffle(data)
split = int(0.8 * len(data))
data_tr, data_te = data[:split], data[split:]

F = data.n_node_features  # Dimension of node features
S = data.n_edge_features  # Dimension of edge features
n_out = data.n_labels  # Dimension of the target
# Data loaders
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(data_te, batch_size=batch_size)

c:\Users\saif8\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Successfully loaded PROTEINS.


c:\Users\saif8\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\saif8\AppData\Local\Temp\ipykernel_21556\3797974380.py:18: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(data)


In [33]:
################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(32, activation="relu")
        self.conv2 = GCNConv(32, activation="relu")
        self.global_pool = GlobalAvgPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, i = inputs
        print(x, a, i)
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


In [34]:
model = Net()
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss="categorical_crossentropy")

In [35]:
################################################################################
# Fit model
################################################################################
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

Epoch 1/10
Tensor("net_2/Cast:0", shape=(None, 4), dtype=float32) SparseTensor(indices=Tensor("DeserializeSparse:0", shape=(None, 2), dtype=int64), values=Tensor("net_2/Cast_1:0", shape=(None,), dtype=float32), dense_shape=Tensor("DeserializeSparse:2", shape=(2,), dtype=int64)) Tensor("IteratorGetNext:2", shape=(None,), dtype=int64)
Tensor("net_2/Cast:0", shape=(None, 4), dtype=float32) SparseTensor(indices=Tensor("DeserializeSparse:0", shape=(None, 2), dtype=int64), values=Tensor("net_2/Cast_1:0", shape=(None,), dtype=float32), dense_shape=Tensor("DeserializeSparse:2", shape=(2,), dtype=int64)) Tensor("IteratorGetNext:2", shape=(None,), dtype=int64)
 9/28 [========>.....................] - ETA: 0s - loss: 9.3463 

c:\Users\saif8\miniconda3\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


28/28 [==============================] - 2s 23ms/step - loss: 9.2905
Epoch 2/10
28/28 [==============================] - 1s 23ms/step - loss: 9.2905
Epoch 3/10
28/28 [==============================] - 1s 24ms/step - loss: 9.2905
Epoch 4/10
28/28 [==============================] - 0s 17ms/step - loss: 9.2905
Epoch 5/10
28/28 [==============================] - 1s 22ms/step - loss: 9.2905
Epoch 6/10
28/28 [==============================] - 0s 16ms/step - loss: 9.2905
Epoch 7/10
28/28 [==============================] - 0s 16ms/step - loss: 9.2905
Epoch 8/10
28/28 [==============================] - 0s 13ms/step - loss: 9.2905
Epoch 9/10
28/28 [==============================] - 0s 16ms/step - loss: 9.2905
Epoch 10/10
 8/28 [=======>......................] - ETA: 0s - loss: 9.2553

In [25]:
model.summary()

Model: "net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gcn_conv (GCNConv)          multiple                  160       
                                                                 
 gcn_conv_1 (GCNConv)        multiple                  1056      
                                                                 
 global_avg_pool (GlobalAvgP  multiple                 0         
 ool)                                                            
                                                                 
 dense (Dense)               multiple                  66        
                                                                 
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


In [22]:
################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(loss))

Testing model
7/7 [==============================] - 0s 13ms/step - loss: 0.6660
Done. Test loss: 0.6660286784172058


In [23]:
loader_tr.tf_signature()

((TensorSpec(shape=(None, 4), dtype=tf.float64, name=None),
  SparseTensorSpec(TensorShape([None, None]), tf.float64),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))

In [27]:
data[0]

Graph(n_nodes=79, n_node_features=4, n_edge_features=None, n_labels=2)

In [7]:
import tensorflow as tf
a = tf.random.normal((1, 3)) # shape = (batch, embedding_1_dim)
b = tf.random.normal((5, 3, 7)) # shape = (output_dim, embedding_1_dim, embedding_2_dim)
c = tf.random.normal((1, 7)) # shape = (batch, embedding_2_dim)
bi = tf.einsum('ik,jkl,il->ij', a, b, c)

In [8]:
bi

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-0.39689445, -1.0620351 ,  6.173465  , -8.726854  , -7.693402  ]],
      dtype=float32)>